In [2]:
# Importing all the necessary libraries -----------------------------------------------------------------------------------

from math import sqrt
import random
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import ipywidgets as widget
from IPython.display import display


# Function to get point for each pixel ------------------------------------------------------------------------------------

def get_points(image_path):  
  img = Image.open(image_path)
  img.thumbnail((200, 400))
  img = img.convert("RGB")
  w, h = img.size  # w,h - dimensions of the image - width and height respectively
  
  points = []
  for count, color in img.getcolors(w * h):  # getcolors() returns the number of pixels of a certain (r,g,b) color, w*h argument is for the maximum number of colors in an image
    for _ in range(count): # _ is the counter for the number of pixels of one color
      points.append(Point(color))
    
  return points  # points is a list of rgb values [(r,g,b), (r,g,b)...]


# Function for getting euclidean distance ---------------------------------------------------------------------------------

def euclidean(p, q):  # takes two points as the input argument
  n_dim = len(p.coordinates)  # coordinate attribute of the point class is just the rgb value of the point
  return sqrt(sum([
      (p.coordinates[i] - q.coordinates[i]) ** 2 for i in range(n_dim)  # this calculates the difference between rgb values of two points
  ]))


# Defining a point class --------------------------------------------------------------------------------------------------

class Point:
  
  def __init__(self, coordinates):
    self.coordinates = coordinates


# Define the cluster class ------------------------------------------------------------------------------------------------

class Cluster:
  
  def __init__(self, center, points):
    self.center = center
    self.points = points


# Define the KMeans class -------------------------------------------------------------------------------------------------

class KMeans:
  
  # 1. initializer for KMeans class
  
  def __init__(self, n_clusters, min_diff = 3):  # putting min_diff to 3 by default (minimum difference between two centers)
    self.n_clusters = n_clusters
    self.min_diff = min_diff
    init : 'kmeans++'


  # 2. this function returns average r,g,b value for an image
  
  def calculate_center(self, points):    
    n_dim = len(points[0].coordinates)   # this will be a Point class with attribute coordinates as (r,g,b) 
    vals = [0.0 for i in range(n_dim)]   # 3 0s 
    for p in points:                     # for all pixels in an image
      for i in range(n_dim):             
        vals[i] += p.coordinates[i]      # [sum of r, sum of g, sum of b]
    coords = [(v / len(points)) for v in vals]  # [avg r, avg g, avg b]
    return Point(coords)  # make the list into a point with coordinates as the average r,g,b in an image
  
  
  # 3. this function assigns points to clusters and returns a list of points arranged in clusters
  
  def assign_points(self, clusters, points):
    point_lists = [[] for i in range(self.n_clusters)] 
    
    # p for point
    for p in points:                     # for each point in image (find the distance between that point and cluster center)
      smallest_distance = float('inf')   # gives the largest possible value for comparison 

      for i in range(self.n_clusters):   # for each cluster
        distance = euclidean(p, clusters[i].center)  
        if distance < smallest_distance:
          smallest_distance = distance
          idx = i

      point_lists[idx].append(p)    # point_lists = [[p1,p4,p5], [p2,p3], [p6,p7..]..]
      # the point gets appended to the list at the index which corresponds to the cluster number with which the point has the smallest distance
    
    return point_lists
    
  def fit(self, points):
    clusters = [Cluster(center=p, points=[p]) for p in random.sample(points, self.n_clusters)] # [cluster obj, cluster obj..] number of item in list = no. of clusters
    # each cluster gets assigned a center point p and a list points of the same point p from all the points in an image
    
    while True:

      point_lists = self.assign_points(clusters, points) # makes the point lists for each cluster within a point_list

      diff = 0

      for i in range(self.n_clusters):  # for all the clusters
        if not point_lists[i]:          # if the cluster does not have any points
          continue
        old = clusters[i]               # old is assigned a cluster from the cluster list
        center = self.calculate_center(point_lists[i])  # center is the center of all points in that cluster
        new = Cluster(center, point_lists[i])           # new is a cluster class with center and all the points
        clusters[i] = new                          # replace the randomly initialized cluster class in the clusters list with the new calculated cluster class
        diff = max(diff, euclidean(old.center, new.center)) # calculate the difference between the center points of the two clusters

      if diff < self.min_diff:
        break

    return clusters # returns the updated clusters list


# Function to convert rgb values to hex -----------------------------------------------------------------------------------

def rgb_to_hex(rgb):
  return '#%s' % ''.join(('%02x' % p for p in rgb))


# Function to get colors from the image -----------------------------------------------------------------------------------

def get_colors(filename, n_colors=8):  # Default number of colors extracted
  jpg = Image.open(filename)
  points = get_points(filename)
  clusters = KMeans(n_clusters=n_colors).fit(points)
  clusters.sort(key=lambda c: len(c.points), reverse = True)
  rgbs = [map(int, c.center.coordinates) for c in clusters]
  color_list = list(map(rgb_to_hex, rgbs))  # convert rgb colors to hex and append them in the list
  return color_list



# Function to convert hex values to rgb (specifically implemented for plotting colors) ------------------------------------

def hex_to_rgb(value):  # value - the color to be converted to rgb
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)) # the tuple contains r, g, b values of color


# Function to convert rgb values to cmyk

def rgb_to_cmyk(rgb_colors):  # rgb_colors is list of rgb values [[r,g,b], [r,g,b]...]
    val = []
    cmyk = []
    
    for i in rgb_colors:
        k = 255
        for j in i:
            j = 1-j/255
            k = min(j,k)
        for j in i:
            val.append((j-k)/(1-k))
        val.append(k)
        cmyk.append(val)
    return cmyk   # cmyk is list of cmyk values [[c,m,y,k], [c,m,y,k]...]


# Function to process the image -------------------------------------------------------------------------------------------

def process(no, filename):
    colors = get_colors("images/"+filename, no) # colors contains the list of colors in hex

    rgb_colors = []  # list to store the colors in rgb format

    for i in colors:  # i - for all colors in list 'colors'
        rgb_colors.append(hex_to_rgb(i))

    x = []
    for i in range(no):
      x.append(i)

    y = [0]
    ylabel = ["Colors"]

    # plotting the rgb colors as images
    plt.imshow([rgb_colors], aspect='equal')
    plt.xticks(x,colors,rotation ='vertical')
    plt.yticks(y,ylabel,rotation ='vertical' )
    plt.savefig("tests/"+filename[0:len(filename)-4]+"palette.jpg")

    # Calculating the percentage of R,G,B colors
    sum_r = 0
    sum_g = 0
    sum_b = 0

    for i in rgb_colors:
        sum_r += i[0]
        sum_g += i[1]
        sum_b += i[2]

    total = sum_r + sum_g + sum_b
    per_r = str(round(sum_r/total*100,2))
    per_g = str(round(sum_g/total*100,2))
    per_b = str(round(sum_b/total*100,2))

    msg = str("RGB    : Red - " + per_r + "%"+"   Green - " + per_g + "%   Blue - " + per_b + "%")

    # Calculating the percentage of C,M,Y,K colors
    cmyk_colors = rgb_to_cmyk(rgb_colors)
    sum_c = 0
    sum_m = 0
    sum_y = 0
    sum_k = 0

    for i in cmyk_colors:
        sum_c += i[0]
        sum_m += i[1]
        sum_y += i[2]
        sum_k += i[3]

    total = sum_c + sum_m + sum_y + sum_k
    per_c = str(round(sum_c/total*100,2))
    per_m = str(round(sum_m/total*100,2))
    per_y = str(round(sum_y/total*100,2))
    per_k = str(round(sum_k/total*100,2))

    msg1 = str("CMYK :  Cyan - " + per_c + "%"+"   Magenta - " + per_m + "%   Yellow - " + per_y + "%   K-Black - " + per_k + "%")

    # merging the original image and generated colors into a single image
    img1 = Image.open("images/"+filename)
    img2 = Image.open("tests/"+filename[0:len(filename)-4]+"palette.jpg")

    img1_size = img1.size
    img2_size = img2.size

    if (img1_size[0] > img1_size[1]):
        img1 = img1.resize((1100, 850))
        img1_size = img1.size
        
    else:
        img1 = img1.resize((500, 750))
        img1_size = img1.size
        
    img2 = img2.resize((img2_size[0], img2_size[0]))
    img2 = img2.rotate(270)
    img2 = img2.resize((500, img1_size[1]))

    new_image = Image.new('RGB',(img1_size[0]+img2_size[0], img1_size[1]+200), (255,255,255))
    new_image.paste(img2,(0,0))
    new_image.paste(img1,(img2_size[0],0))

    draw = ImageDraw.Draw(new_image)
    myFont = ImageFont.truetype('C:/Windows/Fonts/Calibri.ttf', 30)

    if (img1_size[0] > img1_size[1]):
        draw.text((img2_size[0],img1_size[1]+50), msg , fill=(0,0,0), font = myFont)
        draw.text((img2_size[0],img1_size[1]+100), msg1 , fill=(0,0,0), font = myFont)

    else:
        myFont = ImageFont.truetype('C:/Windows/Fonts/Calibri.ttf', 25)
        draw.text((img2_size[0]/2-80,img1_size[1]+50), msg , fill=(0,0,0), font = myFont)
        draw.text((img2_size[0]/2-80,img1_size[1]+90), msg1 , fill=(0,0,0), font = myFont)
    
    new_image.save("tests/"+filename[0:len(filename)-4]+"_merged_image.jpg","JPEG")
    new_image.show()


# Interactive Widgets ----------------------------------------------------------------------------------------------------

slider = widget.IntSlider(
    min=3,
    max=8,
    step=1,
    value=6
)


imgtext = widget.Text()

def btn_eventhandler(b):
    no = slider.value
    filename = imgtext.value
    process(no, filename)

# upload the name of any image from '/images' folder with extension
label1 = widget.Label('Enter the Image Name (with extension):')
label2 = widget.Label('Choose the number of colors:')
btn = widget.Button(description='Submit')

display(label1)
display(imgtext)
display(label2)
display(slider)
display(btn)

btn.on_click(btn_eventhandler)


Label(value='Enter the Image Name (with extension):')

Text(value='')

Label(value='Choose the number of colors:')

IntSlider(value=6, max=8, min=3)

Button(description='Submit', style=ButtonStyle())